<a href="https://colab.research.google.com/github/kodeu/trading-lstm/blob/main/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Example data: 100 data points, 5 features
X = np.random.random((100, 10, 5))  # 100 sequences, 10 time steps per sequence, 5 features
y = np.random.random((100, 1))      # Target variable

print("X var")
print(X[:5])

# Build the LSTM model
model = Sequential([
    LSTM(50, input_shape=(10, 5)),  # 50 LSTM units, 10 time steps, 5 features each step
    Dense(1)                       # Output layer, predicting one value
])

# Compile the model
#model.compile(optimizer='adam', loss='mse')

# Fit the model
#model.fit(X, y, epochs=20, batch_size=1)


X var
[[[0.84735415 0.96474923 0.20622754 0.61348274 0.90241036]
  [0.73208517 0.60125931 0.14470846 0.87736497 0.45576489]
  [0.85187066 0.36235962 0.92125042 0.54080452 0.6431247 ]
  [0.2133932  0.35341419 0.91143929 0.47364826 0.61131851]
  [0.2741732  0.05934346 0.2691215  0.00617203 0.68420291]
  [0.03180913 0.85683197 0.71924531 0.66176876 0.54616098]
  [0.47328631 0.87679091 0.51247305 0.04484821 0.57804854]
  [0.41508173 0.25075108 0.7482694  0.22121309 0.69592326]
  [0.61668526 0.16758549 0.56635136 0.68931852 0.39528449]
  [0.54163887 0.2737599  0.86423515 0.54560132 0.98092686]]

 [[0.62919728 0.78191398 0.69628464 0.13690142 0.35101299]
  [0.72207753 0.30638813 0.96486645 0.12924175 0.46162097]
  [0.12827089 0.63350125 0.86191557 0.08851617 0.77551968]
  [0.59927513 0.99056334 0.85837896 0.52565848 0.60860976]
  [0.9245842  0.41812766 0.69358121 0.73650082 0.82864169]
  [0.26824393 0.79509208 0.94158631 0.92520852 0.9402791 ]
  [0.19604043 0.04553375 0.84830167 0.03181968 0

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Dense, TimeDistributed, Attention
from tensorflow.keras.models import Model

# Assume X and y are prepared as in the previous examples

# Build model with attention
input_layer = Input(shape=(None, 5))
lstm_layer = LSTM(50, return_sequences=True)(input_layer)
attention_layer = Attention()([lstm_layer, lstm_layer])  # Self-attention
output_layer = TimeDistributed(Dense(1))(attention_layer)

print("input layer")
print(lstm_layer[:5])

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='mse')
#model.summary()


input layer
KerasTensor(type_spec=TensorSpec(shape=(None, None, 50), dtype=tf.float32, name=None), name='tf.__operators__.getitem_2/strided_slice:0', description="created by layer 'tf.__operators__.getitem_2'")
